In [90]:
"""
Mark Jay 코드 기반으로 함.
Notebook for streaming data from a microphone in realtime

audio is captured using pyaudio
then converted from binary data to ints using struct
then displayed using matplotlib

scipy.fftpack computes the FFT

if you don't have pyaudio, then run

>>> pip install pyaudio

note: with 2048 samples per chunk, I'm getting 20FPS
when also running the spectrum, its about 15FPS
"""

import pyaudio
import os
import struct
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft
import time
from tkinter import TclError

# to display in separate Tk window
%matplotlib tk

# constants
CHUNK = 1024 * 2      # samples per frame
FORMAT = pyaudio.paInt16     # audio format (bytes per sample?)
CHANNELS = 1                 # single channel for microphone
RATE = 44100                 # samples per second

In [91]:
x = np.arange(0, 2 * CHUNK, 2)  # 0 부터 4094까지 2간격으로 

xf = np.linspace(0, RATE, CHUNK) # 0 부터 RATE 까지 CHUCK간격으로 
x, len(x), xf, len(xf)

(array([   0,    2,    4, ..., 4404, 4406, 4408]),
 2205,
 array([0.00000000e+00, 2.00090744e+01, 4.00181488e+01, ...,
        4.40599819e+04, 4.40799909e+04, 4.41000000e+04]),
 2205)

In [92]:
# create matplotlib figure and axes
fig, (ax1, ax2) = plt.subplots(2, figsize=(15, 7))

# pyaudio class instance
p = pyaudio.PyAudio()

# stream object to get data from microphone
stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    output=True,
    frames_per_buffer= CHUNK
)

# variable for plotting
x = np.arange(0, 2 * CHUNK, 2)       # samples (waveform) arange(처음숫자, 끝숫자, 간격 숫자)
xf = np.linspace(0, RATE, CHUNK)     # frequencies (spectrum) linspace(시작숫자, 끝숫자, 원소 갯수)

# create a line object with random data 실시간 plot을 위한 line 함수들 
line, = ax1.plot(x, np.random.rand(CHUNK), '-', lw=2)  # CHUNK갯수 만큼의 rand(0~1 사이의 랜덤 숫자 생성) 행렬을 만든다. 

# create semilogx line for spectrum
# line_fft, = ax2.semilogx(xf, np.random.rand(CHUNK), '-', lw=2)  #log10(xf)로 변환.
# line_fft, = ax2.semilogy(xf, np.random.rand(CHUNK), '-', lw=2)  #log10(y)를 변환?
# line_fft, = ax2.loglog(xf, np.random.rand(CHUNK), '-', lw=2)  # x,y 모두 log10 scale

line_fft, = ax2.plot(xf, np.random.rand(CHUNK), '-', lw=2)  # 그냥 


# format waveform axes
ax1.set_ylim(0, 255)
ax1.set_xlim(0, 2 * CHUNK)

# format spectrum axes

ax2.set_xlim(20, RATE / 2)
# ax2.set_ylim(0, 200000)       # 새로 추가 

while True:
    
    # binary data
    data = stream.read(CHUNK)  
    
    # convert data to integers, make np array, then offset it by 127
    data_int = struct.unpack(str(2 * CHUNK) + 'B', data)
    
    # create np array and offset by 128
    # data_np = np.array(data_int, dtype='b')[::2] + 128 

    data_np = np.array(data_int, dtype='b')[::2] + 128 
    
    line.set_ydata(data_np) # line(ax1)dp y데이터 입력 
    
    # compute FFT and update line
    yf = fft(data_int)
    line_fft.set_ydata(np.abs(yf[0:CHUNK])  / (128 * CHUNK))  # line_fft(ax2)에 y데이터 입력 
    # line_fft.set_ydata(np.abs(yf[0:CHUNK])  / 1 )             # line_fft(ax2)에 y데이터 입력 
    # update figure canvas
    
    fig.canvas.draw()
    fig.canvas.flush_events()



TclError: invalid command name "pyimage300"